# Классификация текстов с использованием Наивного Байесовского Классификатора

## Задание 1 (1 балл)

Откройте данные. Узнайте, сколько в них спам- и не спам-писем. Визуализируйте полученные соотношение подходящим образом.

In [68]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB, ComplementNB

In [28]:
# откройте данные: ваш код здесь
data = pd.read_csv('spam_or_not_spam.csv')
data.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [84]:
data.groupby('label').count()

,email
label,
0,2500
1,499


In [81]:
data.value_counts().sum()

2999

In [5]:
data.isnull().sum()

email    1
label    0
dtype: int64

In [82]:
data.shape[1]

2

In [ ]:
# рассчитайте частоты для классов : ваш код здесь

In [ ]:
# визуализируйте результат: ваш код здесь

## Задание 2 (2 балла)

Вам необходимо предобработать ваши данные и перевести их в векторный вид. Подгрузим необходимый модуль:

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

Замените в данных все пустые строки и строки, состоящие из пробелов, на пропуски (NaN). После этого удалите из данных все строки, в которых наблюдаются пропущенные значения.

In [30]:
data_clean = data.copy()
data_clean["email"] = data_clean["email"].str.replace(r"\W+", " ", regex=True).str.lower()

data_clean['email'] = data_clean['email'].str.replace('\s+', ' ', regex=True).str.strip()
data_clean['email'] = data_clean['email'].str.lower()
data_clean.head()

,email,label
0,date wed number aug number number number numbe...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which h...,0


In [55]:
data_clean['email'] = data_clean['email'].replace(r'^\s*$', np.NaN, regex=True)

In [56]:
data_clean = data_clean.dropna()

In [57]:
display(data_clean[data_clean['email'] == ''])

,email,label


Переводим данные в векторный вид:

In [58]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data_clean["email"])

Определите, сколько теперь признаков в нашем наборе данных:

In [60]:
#ваш код здесь
print(X.shape, y.shape)

(2997, 34116) (2997,)


## Задание 3 (2 балла)

Определите целевую переменную и признаки:

In [ ]:
#ваш код здесь
y = data_clean["label"]


Разделите выборку на обучающую и тестовую, размер тестовой выборки возьмите как 0.25, параметр random_state определите со значением 42:

In [62]:
#ваш код здесь
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

Рассчитайте среднее значение целевой переменной по тестовой выборке:

In [63]:
#ваш код здесь
print(y_test.mean())

0.16533333333333333


## Задание 4 (3 балла)

Определите и обучите подходящий алгоритм с параметром alpha = 0.01

In [64]:
alpha = 0.01

In [70]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

y_test_pred = clf.predict(X_test)

print(f"Метрики: {metrics.classification_report(y_test, y_test_pred)}")


Метрики:               precision    recall  f1-score   support

           0       0.99      1.00      0.99       626
           1       0.99      0.94      0.97       124

    accuracy                           0.99       750
   macro avg       0.99      0.97      0.98       750
weighted avg       0.99      0.99      0.99       750



In [78]:
clf_1 = ComplementNB()
clf_1.fit(X_train, y_train)

y_test_pred = clf.predict(X_test)

print(f"Метрики: {metrics.classification_report(y_test, y_test_pred)}")

Метрики:               precision    recall  f1-score   support

           0       0.99      1.00      0.99       626
           1       0.99      0.94      0.97       124

    accuracy                           0.99       750
   macro avg       0.99      0.97      0.98       750
weighted avg       0.99      0.99      0.99       750



Оцените результат с точки зрения всех известных вам метрик (не менее трёх):

In [79]:
#ваш код здесь
print('Accuracy: {:.2f}'.format(metrics.accuracy_score(y, y_test_pred)))
#Рассчитываем precision
print('Precision: {:.2f}'.format(metrics.precision_score(y, y_test_pred)))
#Рассчитываем recall
print('Recall: {:.2f}'.format(metrics.recall_score(y, y_test_pred)))
#Рассчитываем F1-меру
print('F1 score: {:.2f}'.format(metrics.f1_score(y, y_test_pred)))

ValueError: Found input variables with inconsistent numbers of samples: [2997, 750]

Нарисуйте ROC-кривую:

In [76]:
y_proba_pred = y_test_pred = clf.predict(X_test)
#Для удобства завернём numpy-массив в Pandas Series
y_proba_pred = pd.Series(y_proba_pred)
#Создадим списки, в которых будем хранить значения метрик 
recall_scores = []
precision_scores = []
f1_scores = []
#Сгенерируем набор вероятностных порогов в диапазоне от 0.1 до 1
thresholds = np.arange(0.1, 1, 0.05)
#В цикле будем перебирать сгенерированные пороги
for threshold in thresholds:
    #Пациентов, для которых вероятность наличия диабета > threshold, относим к классу 1
    #В противном случае — к классу 0
    y_pred = y_proba_pred.apply(lambda x: 1 if x>threshold else 0)
    #Считаем метрики и добавляем их в списки
    recall_scores.append(metrics.recall_score(y, y_pred))
    precision_scores.append(metrics.precision_score(y, y_pred))
    f1_scores.append(metrics.f1_score(y, y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [2997, 750]

## Задание 5 (3 балла)

Переберите несколько значений alpha с помощью кросс-валидации. Оцените, зависит ли от этого параметра качество классификации.

In [ ]:
#ваш код здесь
